<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=99b5e1b9f39499ff663c6b29366387bbbdf45876875d4d6380823ddbee9488a4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 10:45:11
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 19.75 K (0.14%)
Current PnL: -29.12 L (-18.81%)
CY Booked + Current PnL: -14.83 L (-9.58%)
-------------------
Total profit:  1.22 L
Total loss:  -30.34 L
-------------------
Total Booked + Current PnL: 12.15 L (9.51%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.75 L (67.37%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.28,-17.63,21.51,0.09,17857.0,-17762.0,83015.0,92.74,36.0,M-SC,2.34,253.0,-0.99,0.62,9.06,OX40N,NTT,DURABLES
43,ITC,452.00,-0.22,-1.72,12.37,10.44,24332.0,-3438.0,196700.0,-41.15,45.0,X-LC,1.76,5.0,-0.14,1.47,3.75,X40,NTT,FMCG
50,MASFIN,398.61,-0.59,-2.80,25.57,22.05,24352.0,-2745.0,95235.0,-15.99,54.0,H-SC,7.06,164.0,-0.11,0.71,38.43,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.39,-14.07,16.40,0.02,24598.0,-24569.0,149990.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.47,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,0.43,5.34,13.62,19.69,27511.0,10248.0,201990.0,-4.95,47.0,X-MC,2.56,78.0,0.37,1.51,12.51,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,3.96,-17.54,47.44,21.58,54822.0,-24579.0,115560.0,-58.46,27.0,X-MC,7.12,56.0,-0.45,0.86,3.96,X40N,ATH,IT
67,SONACOMS,806.63,3.68,-15.42,64.99,39.54,55604.0,-15601.0,85558.0,-33.83,50.0,M-SC,4.62,220.0,-0.28,0.64,20.64,AR,ATH,AUTO
69,SURYODAY,216.00,2.66,-21.82,57.38,23.03,80329.0,-39076.0,139995.0,53.51,46.0,H-SC,10.10,167.0,-0.49,1.04,38.90,XR,NTT,BANKS
79,VAIBHAVGBL,521.00,2.07,-25.97,117.17,60.77,158536.0,-47471.0,135304.0,55.92,48.0,H-SC,4.12,159.0,-0.30,1.01,21.85,XR,NTT,JEWELLERY
28,GREENPANEL,537.00,1.87,-37.36,128.51,43.14,147375.0,-68398.0,114680.0,157.11,31.0,M-SC,9.50,240.0,-0.46,0.85,7.87,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-1.85,-3.99,26.17,21.14,57123.0,-9064.0,218275.0,-10.72,39.0,X-MC,4.18,74.0,-0.16,1.63,23.03,XY24,NTT,PAINTS
16,CERA,9475.0,-1.54,-31.05,79.69,23.89,96647.0,-54624.0,121279.0,-33.41,31.0,H-SC,9.74,157.0,-0.57,0.90,4.74,OX40N,NTT,CERAMICS
14,CAMPUS,393.0,-1.41,-10.19,48.41,33.28,72940.0,-17104.0,150671.0,-27.64,41.0,M-SC,2.81,222.0,-0.23,1.12,16.78,XY24,NTT,FOOTWEAR
39,INDIGOPNTS,1408.0,-1.39,-14.07,16.40,0.02,24598.0,-24569.0,149990.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.47,OX40N,NTT,PAINTS
70,SYMPHONY,1306.0,-1.33,-33.44,50.20,-0.03,57180.0,-57237.0,113904.0,-46.65,44.0,M-SC,16.80,206.0,-1.00,0.85,1.41,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.15,-7.28,124.24,107.91,170076.0,-10751.0,136893.0,-24.60,37.0,M-SC,12.38,216.0,-0.06,1.02,2.07,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.39,-14.07,16.40,0.02,24598.0,-24569.0,149990.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.47,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.28,-17.63,21.51,0.09,17857.0,-17762.0,83015.0,92.74,36.0,M-SC,2.34,253.0,-0.99,0.62,9.06,OX40N,NTT,DURABLES
66,SIS,528.00,-0.67,-25.38,62.51,21.26,51790.0,-28182.0,82850.0,1952.85,43.0,H-SC,4.61,166.0,-0.54,0.62,11.94,OX40N,NTT,MISC
47,KANSAINER,340.00,-1.06,-25.83,52.99,13.47,105984.0,-69660.0,200007.0,-69.48,29.0,H-SC,4.20,158.0,-0.66,1.49,1.70,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.62,6.47,61.8,72.27,111492.0,10960.0,180408.0,-8.8,67.0,M-LC,1.93,99.0,0.1,1.34,13.04,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.62,6.47,61.80,72.27,111492.0,10960.0,180408.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.34,13.04,XR,NTT,IT
38,INDIAMART,4810.62,0.24,-3.74,114.76,106.72,136243.0,-4616.0,118720.0,-52.34,32.0,H-SC,1.99,139.0,-0.03,0.88,18.76,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.34,-2.28,38.79,35.63,79442.0,-4773.0,204801.0,-15.85,40.0,H-MC,3.14,119.0,-0.06,1.53,14.01,AR,ATH,PHARMA
25,FINCABLES,1641.55,-1.15,-7.28,124.24,107.91,170076.0,-10751.0,136893.0,-24.60,37.0,M-SC,12.38,216.0,-0.06,1.02,2.07,OX40N,ATH,CABLES
37,IEX,219.00,0.92,-4.48,55.60,48.63,106195.0,-8956.0,190998.0,-36.48,47.0,H-SC,14.07,136.0,-0.08,1.42,6.56,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.15,-21.41,88.89,48.45,140080.0,-42918.0,157588.0,-26.96,25.0,X-MC,12.18,64.0,-0.31,1.17,2.61,X40N,ATH,FINANCE
75,TMPV,600.00,0.16,-18.50,74.44,42.18,167704.0,-51123.0,225287.0,-26.74,25.0,X-LC,6.08,3.0,-0.30,1.68,0.16,XY24,NTT,AUTO
22,DIXON,18931.72,3.96,-17.54,47.44,21.58,54822.0,-24579.0,115560.0,-58.46,27.0,X-MC,7.12,56.0,-0.45,0.86,3.96,X40N,ATH,IT
33,HINDUNILVR,2922.00,0.13,-10.34,26.78,13.67,61723.0,-26583.0,230480.0,-17.11,29.0,X-LC,2.63,14.0,-0.43,1.72,8.81,XY25,NTT,FMCG
53,PAGEIND,51605.08,0.59,-9.83,39.81,26.07,58775.0,-16100.0,147640.0,-33.64,30.0,X-MC,12.15,55.0,-0.27,1.10,0.59,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,1.72,-80.35,535.02,24.79,264797.0,-202363.0,49493.0,-81.32,42.0,X-SC,1.23,86.0,-0.76,0.37,20.99,X40N,NTT,MISC
8,AWL,485.0,0.02,-21.76,96.20,53.50,227105.0,-65675.0,236076.0,-62.10,33.0,X-MC,1.71,58.0,-0.29,1.76,4.86,XY24,NTT,FMCG
43,ITC,452.0,-0.22,-1.72,12.37,10.44,24332.0,-3438.0,196700.0,-41.15,45.0,X-LC,1.76,5.0,-0.14,1.47,3.75,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.69,5.84,23.75,30.97,44425.0,10313.0,187051.0,-19.08,61.0,X-MC,2.04,75.0,0.23,1.39,19.13,X40,ATH,INSURANCE
3,ACC,3906.0,-0.18,-25.23,119.55,64.15,212691.0,-60041.0,177910.0,-56.31,34.0,X-SC,2.10,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.18,-25.23,119.55,64.15,212691.0,-60041.0,177910.0,-56.31,34.0,X-SC,2.10,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.53,-6.39,42.88,33.76,80612.0,-12825.0,187995.0,-33.99,32.0,X-MC,5.25,57.0,-0.16,1.40,0.00,X40,ATH,FMCG
75,TMPV,600.00,0.16,-18.50,74.44,42.18,167704.0,-51123.0,225287.0,-26.74,25.0,X-LC,6.08,3.0,-0.30,1.68,0.16,XY24,NTT,AUTO
31,HAVELLS,2069.16,0.34,-10.71,48.37,32.48,141659.0,-35125.0,292866.0,-15.19,36.0,X-MC,5.51,67.0,-0.25,2.18,0.34,X40,ATH,ELECTRICAL
53,PAGEIND,51605.08,0.59,-9.83,39.81,26.07,58775.0,-16100.0,147640.0,-33.64,30.0,X-MC,12.15,55.0,-0.27,1.10,0.59,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.29,-30.01,104.10,42.84,47360.0,-19511.0,45495.0,-54.27,41.0,X-SC,36.54,83.0,-0.41,0.34,2.93,XY24,NTT,MISC
15,CAMS,4762.00,1.72,-80.35,535.02,24.79,264797.0,-202363.0,49493.0,-81.32,42.0,X-SC,1.23,86.0,-0.76,0.37,20.99,X40N,NTT,MISC
59,RELAXO,1176.00,-1.14,-47.29,188.94,52.31,144569.0,-68644.0,76516.0,-43.86,44.0,X-SC,6.42,91.0,-0.47,0.57,2.42,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.02,-37.91,117.74,35.20,94070.0,-48774.0,79896.0,4.69,34.0,X-SC,15.67,92.0,-0.52,0.60,1.49,X40,NTT,FOOTWEAR
22,DIXON,18931.72,3.96,-17.54,47.44,21.58,54822.0,-24579.0,115560.0,-58.46,27.0,X-MC,7.12,56.0,-0.45,0.86,3.96,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.43,-13.72,38.24,19.28,114149.0,-47450.0,298506.0,-24.92,56.0,X-LC,5.22,1.0,-0.42,2.22,10.35,X40,ATH,IT
41,INFY,2275.00,0.27,8.18,43.16,54.87,146087.0,25590.0,338478.0,-15.65,60.0,X-LC,3.21,2.0,0.18,2.52,17.14,X40,BTT,IT
75,TMPV,600.00,0.16,-18.50,74.44,42.18,167704.0,-51123.0,225287.0,-26.74,25.0,X-LC,6.08,3.0,-0.30,1.68,0.16,XY24,NTT,AUTO
81,VBL,671.64,-0.55,-4.99,42.80,35.67,128436.0,-15759.0,300083.0,-16.24,52.0,X-LC,4.61,4.0,-0.12,2.24,8.17,X40N,ATH,FMCG
43,ITC,452.00,-0.22,-1.72,12.37,10.44,24332.0,-3438.0,196700.0,-41.15,45.0,X-LC,1.76,5.0,-0.14,1.47,3.75,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.02,-17.77,120.93,81.67,93285.0,-16670.0,77140.0,6933.33,40.0,L-SC,13.85,271.0,-0.18,0.57,49.39,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.59,-2.80,25.57,22.05,24352.0,-2745.0,95235.0,-15.99,54.0,H-SC,7.06,164.0,-0.11,0.71,38.43,XR,ATH,FINANCE
13,BSOFT,831.70,0.67,-22.78,95.26,50.79,103469.0,-32036.0,108617.0,-1.04,66.0,H-SC,5.84,151.0,-0.31,0.81,26.47,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.39,-14.07,16.40,0.02,24598.0,-24569.0,149990.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.47,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.30,-1.97,23.38,20.95,57701.0,-4972.0,246796.0,-3.02,50.0,X-LC,19.00,31.0,-0.09,1.84,32.96,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.67,-22.78,95.26,50.79,103469.0,-32036.0,108617.0,-1.04,66.0,H-SC,5.84,151.0,-0.31,0.81,26.47,XR,ATH,IT
84,WIPRO,420.00,0.62,6.47,61.80,72.27,111492.0,10960.0,180408.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.34,13.04,XR,NTT,IT
56,RAJESHEXPO,518.00,0.83,-62.95,170.06,0.07,86441.0,-86347.0,50830.0,1762.37,57.0,L-SC,2.38,268.0,-1.00,0.38,26.42,OX40N,NTT,JEWELLERY
32,HCLTECH,1922.01,-0.23,7.86,15.64,24.73,40813.0,19026.0,260950.0,12.76,62.0,X-LC,6.18,7.0,0.47,1.94,24.30,X40,ATH,IT
65,SIEMENS,4671.50,-0.06,-14.61,46.99,25.51,74667.0,-27195.0,158900.0,-2.45,46.0,H-LC,4.14,49.0,-0.36,1.18,17.10,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.71
1,25,45.76
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.29
LC    24.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.29
X40      22.42
X40N     10.82
XR       10.11
XY25      9.42
AR        9.39
OX40N     7.77
SR        1.02
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.55
X-MC    23.51
X-LC    21.07
M-SC    12.11
X-SC     6.97
H-MC     5.02
M-MC     1.41
L-SC     1.41
M-LC     1.34
H-LC     1.18
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.96,42.88
IT,12.74,-17.74,81.39
FINANCE,9.60,-20.26,71.08
ELECTRICAL,6.23,-12.08,53.14
MISC,6.09,-58.74,123.77
PAINTS,6.08,-13.28,30.11
INSURANCE,4.61,-2.01,37.96
PHARMA,4.12,-1.00,33.33
AUTO,3.42,-25.35,79.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3238640.0,21
AR,1312345.0,10
XR,1302738.0,13
X40,1027394.0,14
X40N,996512.0,9
OX40N,750858.0,10
XY25,378372.0,6
SR,283424.0,2
MH,84626.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3682794.0,25
M-SC,1432283.0,15
X-MC,1326978.0,16
X-SC,985579.0,8
X-LC,935488.0,11
H-MC,405668.0,3
L-SC,271519.0,3
M-LC,111492.0,1
H-LC,74667.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1287278.0      6
           AR         901277.0      5
M-SC       XY24       816512.0      6
H-SC       XR         777361.0      7
X-SC       X40N       507396.0      3
X-MC       X40        461173.0      7
           XY24       398103.0      3
X-LC       X40        383082.0      5
H-SC       OX40N      348828.0      4
M-SC       OX40N      315589.0      5
X-SC       XY24       295044.0      3
H-SC       SR         283424.0      2
X-MC       X40N       281899.0      4
X-LC       XY24       246094.0      2
H-MC       AR         210059.0      2
X-LC       X40N       207217.0      2
H-MC       XY24       195609.0      1
X-MC       XY25       185803.0      2
L-SC       XR         185078.0      2
X-SC       X40        183139.0      2
M-SC       XR         173840.0      2
           AR         126342.0      2
M-LC       XR         111492.0      1
X-LC       XY25        99095.0      2
L-SC       OX40N       86441.0      1
H-SC       MH          84626.0      1
H-LC       AR          74667.0      1
M-MC       XY25        56098.0      1
L-MC       XR          54967.0      1
L-LC       XY25        37376.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
